<a href="https://colab.research.google.com/github/Andy-Lewis-Sapner/ITCC-Ant-SemB24/blob/master/Project/Project_Ant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instructions#

- To start the application, please run all of the cells (from start to finish). Running all of them is possible by choosing Runtime -> Run all (or CTRL + F9).
- It may take about 1 to 2 minutes to run all the code, since there are installations of python packages happening when running the code.
- In order to use the application, please go to the Main Program section and open it. It is encouraged to use a full screen of the output, and is possible by choosing the 3 dots on the cell, and then "View output fullscreen".

- The login credentials for this application, in this stage of development are:
```
    username = "testuser"
    password = "testpassword"
```
- After completing the login process, the home screen is shown. It has 5 options, each one for each screen. The database will contain always a json file. You may use the JSON Upload screen to upload your own json file, or for safety, upload the one that is mentioned below.

- The json file used for this project is called: "edited Audit Trail Dashboard (3).json_label".

- Navigation between screens is done by choosing a screen in the navigation bar or choosing home and then a screen. You can use any screen that is available, by navigating to it.

#Installations#

In [1]:
!pip install squarify firebase requests beautifulsoup4 nltk gdown

#Imports#

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime, timedelta
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, GridspecLayout, interact, HBox, VBox
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import squarify
import textwrap
import random
import re
import networkx as nx
from collections import Counter
import json
from firebase import firebase
import requests
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
import nltk
from nltk.chat.util import Chat, reflections
import gdown
from google.colab import files

#Utils#

##DB Functionality##

In [3]:
# Connects to a Firebase database using the provided URL
def connect_to_DB(DB_url):
 return firebase.FirebaseApplication(DB_url, None)

In [4]:
# Saves a JSON file to the Firebase database
def saveJsonToDB(jsonFile):
  FBConn.post('/onShapeJSON/', jsonFile)

In [5]:
# Opens and returns the latest JSON file from the Firebase database
def openJsonFromDB():
  result = FBConn.get('/onShapeJSON/', None)
  return list(result.values())[-1]

##Global Variables##

In [6]:
# Define global variables
# filters_dict is used to store the user-selected filters
filters_dict = {'Graph': 'temp', 'Project Name': 'temp', 'Student Name': None, 'Tab Name': None, 'Start Date': 'date1', 'End Date': 'date2', 'Actions Type': False}
# DB_url is the URL of the Firebase database
DB_url = 'https://onshapeassistant-default-rtdb.firebaseio.com/'
# glossary_url is the URL of the glossary for the index
glossary_url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'
menu_buttons_list = []
data = []
# login_dict is used to store the user's login credentials
login_dict = {'Username': 'temp', 'Password': 'temp'}

##Download Nltk##

In [7]:
# Download nltk resources for tokenization and lemmatization
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

##Index Creation##

In [8]:
# Fetches the content of a webpage given its URL assuming the response is an html document
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

In [9]:
# Indexes words from the HTML content, counting occurrences
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

In [10]:
# Returns a list of common stop words to be excluded from indexing
def get_stop_words():
  words = [
    "a", "able", "about", "above", "abst", "accordance", "according", "accordingly", "across", "act", "actually",
    "added", "adj", "affected", "affecting", "affects", "after", "afterwards", "again", "against", "ah", "all",
    "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "an", "and",
    "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways",
    "anywhere", "apparently", "approximately", "are", "aren", "arent", "arise", "around", "as", "aside", "ask",
    "asking", "at", "auth", "available", "away", "awfully", "b", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind",
    "being", "believe", "below", "beside", "besides", "between", "beyond", "biol", "both", "brief", "briefly", "but",
    "by", "c", "ca", "came", "can", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com",
    "come", "comes", "contain", "containing", "contains", "could", "couldnt", "d", "date", "did", "didn't",
    "different", "do", "does", "doesn't", "doing", "done", "don't", "down", "downwards", "due", "during", "e",
    "each", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough",
    "especially", "et", "et-al", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere",
    "ex", "except", "f", "far", "few", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows",
    "for", "former", "formerly", "forth", "found", "four", "from", "further", "furthermore", "g", "gave", "get",
    "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "had",
    "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he", "hed", "hence", "her", "here",
    "hereafter", "hereby", "herein", "heres", "hereupon", "hers", "herself", "hes", "hi", "hid", "him", "himself",
    "his", "hither", "home", "how", "howbeit", "however", "hundred", "i", "id", "ie", "if", "i'll", "im", "immediate",
    "immediately", "importance", "important", "in", "inc", "indeed", "index", "information", "instead", "into",
    "invention", "inward", "is", "isn't", "it", "itd", "it'll", "its", "itself", "i've", "j", "just", "k", "keep",
    "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter",
    "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "ll", "look",
    "looking", "looks", "ltd", "m", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means",
    "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "more", "moreover", "most", "mostly",
    "mr", "mrs", "much", "mug", "must", "my", "myself", "n", "na", "name", "namely", "nay", "nd", "near", "nearly",
    "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety",
    "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing",
    "now", "nowhere", "o", "obtain", "obtained", "obviously", "of", "off", "often", "oh", "ok", "okay", "old",
    "omitted", "on", "once", "one", "ones", "only", "onto", "or", "ord", "other", "others", "otherwise", "ought",
    "our", "ours", "ourselves", "out", "outside", "over", "overall", "owing", "own", "p", "page", "pages", "part",
    "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible",
    "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly",
    "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "re", "readily",
    "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively",
    "research", "respectively", "resulted", "resulting", "results", "right", "run", "s", "said", "same", "saw",
    "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen",
    "self", "selves", "sent", "seven", "several", "shall", "she", "shed", "she'll", "shes", "should", "shouldn't",
    "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since",
    "six", "slightly", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime",
    "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying",
    "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup",
    "sure", "t", "take", "taken", "taking", "tell", "tends", "th", "than", "thank", "thanks", "thanx", "that",
    "that'll", "thats", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there",
    "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto",
    "thereupon", "there've", "these", "they", "theyd", "they'll", "theyre", "they've", "think", "this", "those",
    "thou", "though", "thoughh", "thousand", "throug", "through", "throughout", "thru", "thus", "til", "tip", "to",
    "together", "too", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two",
    "u", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "up", "upon", "ups", "us",
    "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "ve",
    "very", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "was", "wasnt", "way", "we", "wed", "welcome",
    "we'll", "went", "were", "werent", "we've", "what", "whatever", "what'll", "whats", "when", "whence", "whenever",
    "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which",
    "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "whose",
    "why", "widely", "willing", "wish", "with", "within", "without", "wont", "words", "world", "would", "wouldnt",
    "www", "x", "y", "yes", "yet", "you", "youd", "you'll", "your", "youre", "yours", "yourself", "yourselves",
    "you've", "z", "zero"
  ]
  return words

In [11]:
# Removes stop words from the indexed words
def remove_stop_words(index):
    stop_words = get_stop_words()
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

In [12]:
# Applies stemming to the indexed words to reduce them to their root forms
def apply_stemming(index):
    # Porter Stemmer - the common Stemming Algorithm for the English language
    stemmer = PorterStemmer()
    stemmed_index = {}
    # Stem each word in the index and add it to the stemmed index with its count
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

In [13]:
# Finds and returns the 10 most important words from a webpage, by their occurrences
def find_important_10_words(url):
  soup = fetch_page(url)
  if soup is not None:
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    important10 = dict(sorted(index.items(), key=lambda item: item[1]))
    important10 = list(important10.keys())[-10:]
    return important10, index
  return None

In [14]:
# Creates an index of the top 10 most important words
def create_important_10_index(important10, initial_index):
  index = dict()
  for word in important10:
    index[word] = initial_index[word]
  return index

In [15]:
# Posts the created index (with the 10 most important terms) to the Firebase database
def post_index_to_DB():
  important10, initial_index = find_important_10_words(glossary_url)
  index = create_important_10_index(important10, initial_index)
  FBConn.post('/onshapeIndex/', index)
  index_is_posted = True

In [16]:
# Retrieves the last posted index from the Firebase database
def get_index():
  return list(FBConn.get('/onshapeIndex/', None).values())[-1]

##Styles##

In [17]:
par_style = """
<style>
    .par-screen .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .error {
      font-size: 12px;
      color: white;
      background-color: red;
      margin: 150px auto;
      width: 25%;
      text-align: center;
    }
    .par-screen .title {
      font-size: 20px;
      text-align: center;
      border: 2px outset #d6b86f;
    }
    .par-screen .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 16px auto;
      width: 70%;
      padding: 4px;
    }
    .par-screen {
      max-width: 100%;
    }
    .par-screen .component {
      border: 2px dotted #261e0c;
      padding: 4px;
    }
    .par-screen .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
</style>
"""

In [18]:
index_style = """
<style>
    .index-screen .widget-label {
      border: 2px dotted #f5f0e4;
      font-weight: bold;
      font-size: 12px;
      color: black;
      padding: 0 8px;
      width: 100%;
    }
    .index-screen {
      color: black;
    }
    .screen-title {
      font-size: 32px;
      text-align: center;
      border: 2px outset #d6b86f;
      margin: 20px auto; /* Ensure space below the title */
      width: 100%;
      padding: 10px;
      color: black;
    }
    .index-screen {
      max-width: 100%;
      display: flex;
      flex-direction: column;
      justify-content: flex-start; /* Ensure the title is at the top */
      align-items: center;
      height: 100vh; /* Full viewport height */
      margin: 0 auto;
    }
    .index-screen .widget-button {
      margin: 16px auto;
      background-color: green;
      border-radius: 5px;
    }
    .index-screen .widget-button:hover {
      background-color: darkgreen;
    }
    .index-screen .widget-button:active {
      background-color: #77b300;
    }
    .index-screen .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
    }
    .centered-widget {
      display: flex;
      justify-content: center;
      align-items: center;
      flex-direction: column;
      color: black;
    }
    .index-screen .output {
      color: black; /* Set the text color of the output to black */
    }
</style>
"""

In [19]:
json_style = """
<style>
    # .json-screen .widget-label {
    #   font-weight: bold;
    #   font-size: 12px;
    #   color: black;
    #   padding: 0 8px;
    #   width: 100%;
    # }
    # .screen-title {
    #   font-size: 32px;
    #   text-align: center;
    #   border: 2px outset #d6b86f;
    #   margin: 16px auto;
    #   width: 70%;
    #   padding: 4px;
    #   color: black;
    # }
    # .json-screen {
    #   max-width: 100%;
    #   display: flex;
    #   justify-content: center;
    #   align-items: center;
    #   height: full;
    #   color: black;
    # }
    .json-screen .widget-hbox {
      width: 100%;
      margin: 4px auto;
      height: 100%;
      color: black;  /* Ensure all text in hbox is black */
    }
    .json-screen .centered-widget {
      display: flex;
      justify-content: center;
      align-items: center;
      flex-direction: column;
      color: black;
    }
    .widget-textarea .widget-label,
    .widget-label-basic,
    .widget-label-level,
    .widget-label-output {
        color: black;
    }
</style>
"""

In [20]:
page_style = """
<style>
  .page {
    background-color: #ccd9ff;
    min-height: 875px;
  }
  .home-screen {
    margin: 8px auto;
  }
  .home-screen .buttons-vbox {
    margin-top: 32px;
  }
  .home-screen .widget-hbox {
    margin: 16px 0;
  }
  .home-screen .welcome-label {
    font-size: 40px;
    background: linear-gradient(to right, #f32170, #ff6b08, #cf23cf, #278a41);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    height: 52px;
    margin: 16px auto;
    padding-top: 4px;
  }
  .home-screen .widget-button {
    font-size: 32px;
    height: 300px;
    width: 300px;
    margin: 0 64px;
    background-color: #ccd9ff;
    color: #CFFF00;
    background-repeat: no-repeat;
    background-position: center;
    background-size: 200px;
    border-radius: 50%;
  }
  .home-screen .widget-button:hover {
    background-color: #809fff;
  }
  .home-screen .widget-button:active {
    background-color: #668cff;
  }
  .home-screen .ju {
    background-image: url("https://www.taniarascia.com/static/9d9dd7f6cf3b89757f9b4e17227ee5a5/92ab1/json.png");
  }
  .home-screen .is {
    background-image: url("https://images.pexels.com/photos/4494642/pexels-photo-4494642.jpeg?auto=compress&cs=tinysrgb&w=600");
  }
  .home-screen .ps {
    background-image: url("https://cdn.pixabay.com/photo/2017/05/09/10/03/equalizer-2297756_1280.png");
  }
  .home-screen .sts {
    background-image: url("https://cdn.sanity.io/images/tlr8oxjg/production/c277918894f2a79f6a029ffaecdcf3f22c7281b2-1456x816.png?w=3840&q=80&fit=clip&auto=format");
  }
  .home-screen .cs {
    background-image: url("https://cdn.dribbble.com/userupload/10543013/file/still-ea4dc478539e52662286d4f78a76f56f.gif?resize=400x0");
  }
  .menu .widget-button {
    background-color: #b3c6ff;
    color: black;
    margin: 4px 0 0 0.3em;
  }
  .menu .widget-button:hover {
    font-weight: bold;
  }
  .login-screen .widget-label {
    color: black;
  }
  .login-screen {
    border: 4px solid #002080;
    margin: 0 128px;
  }
  .param-menu .widget-label {
    color: black;
  }
  .statistics-screen .widget-label {
    color: black;
  }
  .statistics-output {
    color: black;
  }
  .statistics-output .widget-label {
    color: black;
  }
</style>
"""

#All Screens#

##Login Screen##

In [21]:
# Creates the login screen UI for the application
def create_login_screen():
  # Create the grid
  grid = GridspecLayout(100, 100, height='400px')
  grid.add_class("login-screen")
  error_label = widgets.Label(value='', layout=Layout(color='red'))

  def on_button_clicked(b):
      login_dict['Username'] = grid[20:27, 38:58].value #= dropdown_graphtype()
      login_dict['Password'] = grid[30:37, 38:58].value #= project_name()
      if login_dict['Username'] == 'testuser' and login_dict['Password'] == 'testpassword':
        page.children = [create_home_screen()]
      else:
        error_label.value = "Login Failed"

  def create_expanded_button(description, button_style):
      button = Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
      return button

  # Place the widgets with height of 7 rows each
  grid[10:20, 48:78] = widgets.Label(value='Login', layout=Layout(color='black'))
  grid[20:27, 38:58] = login_username()
  grid[30:37, 38:58] = login_password()
  grid[43:53, 46:53] = create_expanded_button('Login', 'success')
  grid[10:20, 48:48] = error_label

  grid[43:53, 46:53].on_click(on_button_clicked)
  login_page = widgets.VBox([grid], layout=widgets.Layout(margin='300px auto'))
  return login_page

# Creates a text input widget for the username
def login_username():
  return widgets.Text(
    placeholder='Enter Username',
    description='Username:',
    disabled=False
  )

# Creates a password input widget
def login_password():
  return widgets.Password(
    placeholder='Enter password',
    description='Password:',
    disabled=False
  )

##Home Screen##

In [22]:
# Handles the event when the JSON upload button is clicked, switches the page to the json upload page
def json_upload_clicked(b):
  page.children = [create_menu(), create_json_upload_screen()]

In [23]:
# Handles the event when the Index Search button is clicked, switches the page to the index search page
def index_search_clicked(b):
  page.children = [create_menu(), create_index_search_screen()]

In [24]:
# Handles the event when the Parameters button is clicked, switches the page to the parameters screen page
def parameters_clicked(b):
  page.children = [create_menu(), create_parameters_screen()]

In [25]:
# Handles the event when the Statistics button is clicked, switches the page to the statistics screen page
def statistics_clicked(b):
  page.children = [create_menu(), create_statistics_screen()]

In [26]:
# Creates the home screen UI for the application with buttons
def create_home_screen():
  welcome_label = widgets.Label("Welcome to OnShape Project Manager Assist")
  welcome_label.add_class("welcome-label")

  json_upload_screen = widgets.Button(description="JSON Upload")
  json_upload_screen.add_class("ju")
  json_upload_screen.on_click(json_upload_clicked)

  index_search_screen = widgets.Button(description="Index Search", layout=widgets.Layout(disabled=True))
  index_search_screen.add_class("is")
  index_search_screen.on_click(index_search_clicked)

  parameters_screen = widgets.Button(description="Parameters", layout=widgets.Layout(disabled=True))
  parameters_screen.add_class("ps")
  parameters_screen.on_click(parameters_clicked)

  statistics_screen = widgets.Button(description="Statistics", layout=widgets.Layout(disabled=True))
  statistics_screen.add_class("sts")
  statistics_screen.on_click(statistics_clicked)

  chatbot_screen = widgets.Button(description="Chatbot", layout=widgets.Layout(disabled=True, margin='0 auto'))
  chatbot_screen.add_class("cs")
  chatbot_screen.on_click(chatbot_clicked)

  menu_buttons_list.extend([json_upload_screen, index_search_screen, parameters_screen, statistics_screen])

  hbox1 = widgets.HBox([parameters_screen, statistics_screen])
  hbox2 = widgets.HBox([json_upload_screen, index_search_screen])
  hbox3 = widgets.HBox([chatbot_screen])

  buttons_vbox = widgets.VBox([hbox1, hbox2, hbox3])
  buttons_vbox.add_class("buttons-vbox")
  home_screen = widgets.VBox([welcome_label, buttons_vbox])
  home_screen.add_class("home-screen")
  return home_screen

##Parameters Screen##

###Parameters Menu Screen###

In [27]:
# Creates the parameters screen UI where users can filter data
def create_parameters_screen():
  global documentlist, tablist, userlist
  grid = GridspecLayout(100, 100, height='400px')
  grid.add_class('param-menu')

  def on_button_clicked(b):
      filters_dict['Graph'] = grid[20:27, 20:50].value #= dropdown_graphtype()
      filters_dict['Project Name'] = grid[27:34, 20:50].label #= project_name()
      filters_dict['Studnet Name'] = grid[34:41, 20:50].label #= studet_name()
      filters_dict['Tab Name'] = grid[41:48, 20:50].label #= card_name()
      filters_dict['Start Date'] = grid[48:55, 20:50].value #= start_date_picker()
      filters_dict['End Date'] = grid[48:55, 50:].value #=  end_date_picker()
      page.children = [create_menu(), create_parameters_results_screen(filters_dict, openJsonFromDB())]

  def create_expanded_button(description, button_style):
    button = Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
    return button

  documentlist = set()
  tablist = set()
  userlist = set()
  databaseresults = openJsonFromDB()

  for item in databaseresults:
    if isinstance(item, dict):
      for key, value in item.items():
        if key == 'Document':
          documentlist.add(value)
        elif key == 'Tab':
          tablist.add(value)
        elif key == 'User':
          userlist.add(value)

  # Place the widgets with height of 7 rows each
  grid[10:20, 40:60] = widgets.Label(value='Filter Zone')
  grid[20:27, 20:50] = dropdown_graphtype()
  grid[27:34, 20:50] = project_name()
  grid[34:41, 20:50] = studet_name()
  grid[34:41, 50:] = widgets.Label(value='(optional)')
  grid[41:48, 20:50] = tab_name()
  grid[41:48, 50:] = widgets.Label(value='(optional)')
  grid[48:55, 20:50] = start_date_picker()
  grid[48:55, 50:] = end_date_picker()
  grid[90:, 45:55] = create_expanded_button('Next', 'success')
  grid[90:, 45:55].on_click(on_button_clicked)
  return grid

# Creates a dropdown widget for selecting the graph type
def dropdown_graphtype():
  return widgets.Dropdown(
    options=[('activities_quantity', 'AQ'), ('work_hours', 'WH'), ('relative_wrok', 'RW'), ('common_activities_in_project', 'CA')],
    description='Type:',
    layout=Layout(height='auto', width='auto'),
  )

# Creates a dropdown widget for selecting the project name
def project_name():
  return widgets.Dropdown(
    options=list(documentlist),
    description='Project:',
    layout=Layout(height='auto', width='auto'),
  )

# Creates a dropdown widget for selecting the student (user) name
def studet_name():
  return widgets.Dropdown(
    options=list(userlist),
    description='Student:',
    layout=Layout(height='auto', width='auto'),
  )

# Creates a dropdown widget for selecting the tab name
def tab_name():
  return widgets.Dropdown(
    options=list(tablist),
    description='Tab:',
    layout=Layout(height='auto', width='auto'),
  )

# Creates a date picker widget for selecting the start date
def start_date_picker():
  return widgets.DatePicker(
    description='Start Date:',
    disabled=False
  )

# Creates a date picker widget for selecting the end date
def end_date_picker():
  return widgets.DatePicker(
    description='End Date:',
    disabled=False
  )

###Parameters Results Screen###

In [28]:
# Plots the quantity of activities based on the filtered data
def activities_quantity(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S")

    if filters_dict['Start Date'] is None or filters_dict['End Date'] is None:
        plt.plot()
        return plt

    # Convert the filter dates to datetime format
    start_date = pd.to_datetime(filters_dict['Start Date'])
    end_date = pd.to_datetime(filters_dict['End Date'])

    # Filter the DataFrame based on the 'Project Name'
    df = df[df['Document'] == filters_dict['Project Name']]

    # Filter the DataFrame based on the date range
    df = df[(df['Time'] >= start_date) & (df['Time'] <= end_date)]

    # Check if the date range spans multiple months
    months_range = end_date.month != start_date.month

    # Create the 'activity_time' column based on the date range
    if months_range:
        df['activity_time'] = df['Time'].dt.strftime('%m/%Y')
    else:
        df['activity_time'] = df['Time'].dt.strftime('%d/%m')

    # Group by 'User' and 'activity_time' and count the activities
    activity_counts = df.groupby(['User', 'activity_time']).size().unstack(fill_value=0)

    if activity_counts.empty:
        plt.plot()
        return plt

    # Plot the data
    activity_counts.T.plot(kind='bar', stacked=True)

    plt.xlabel('Date')
    plt.xticks(rotation=90)
    plt.ylabel('Number of activities')
    plt.title('Activities per User in Project "' + filters_dict['Project Name'] + '"')
    plt.legend(title='User')
    plt.tight_layout()

    return plt

In [29]:
# Plots the work hours of users based on the filtered data
def work_hours(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S")

    # Filter the DataFrame based on the 'Project Name'
    df = df[df['Document'] == filters_dict['Project Name']]

    # Filter by 'Student Name' if provided
    if filters_dict['Student Name'] is not None:
        df = df[df['User'] == filters_dict['Student Name']]

    # Get unique users
    users = df['User'].unique()

    points_of_time = []
    for user in users:
        user_df = df[df['User'] == user]

        if user_df.empty:
            continue

        user_work_hours = user_df['Time'].dt.strftime("%H:%M")
        min_time = min(user_work_hours)
        max_time = max(user_work_hours)
        start_time = datetime.strptime(min_time, "%H:%M")
        end_time = datetime.strptime(max_time, "%H:%M")
        time_diff = (end_time - start_time).total_seconds() / 3600  # time_diff in hours
        points_of_time.append([user, start_time, time_diff])

    fig, ax = plt.subplots()
    for user, start_time, time_diff in points_of_time:
        ax.barh(user, time_diff, left=start_time.hour + start_time.minute / 60, label=user)

    # Formatting the x-axis to display time properly
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:02}:{:02}'.format(int(x), int((x % 1) * 60))))

    plt.xlabel('Time of Day')
    plt.ylabel('User')
    plt.title('Work Hours per User in Project "' + filters_dict['Project Name'] + '"')
    plt.legend()
    plt.tight_layout()
    return plt

In [30]:
# Plots the relative work of users on a project tab based on the filtered data
def relative_work(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Filter the DataFrame based on the 'Tab Name' and 'Project Name'
    df = df[(df['Tab'] == filters_dict['Tab Name']) & (df['Document'] == filters_dict['Project Name'])]

    # Check if 'Student Name' filter is applied
    if filters_dict['Student Name'] is not None:
        df['User'] = df['User'].apply(lambda x: 'Others' if x != filters_dict['Student Name'] else x)

    # Count the occurrences of each user
    quantities = df['User'].value_counts()

    # Plot the data as a pie chart
    plt.pie(quantities.values, labels=quantities.index, autopct='%1.1f%%')
    plt.axis('equal')
    plt.title('Relative Work on Tab "' + filters_dict['Tab Name'] + '" in Project "' + filters_dict['Project Name'] + '"')
    plt.tight_layout()

    return plt

In [31]:
# Shortens descriptions to fit within a specified format
def shorten_description(description):
    # Define patterns to shorten
    patterns = [
        (r'\b(Tab [A-Z][a-z]* [0-9]+)', r'\1'),
        (r'\b(Stop|Start) assembly drag', r'\1 drag'),
        (r'\b(Tab [A-Z][a-z]*) \.\.\.', r'\1 is'),
        (r'\b(Tab [A-Z][a-z]*) of type [A-Z]+ (closed|opened) by [A-Za-z]+', r'\1 \2'),
        (r'\b(Tab [A-Z][a-z]*) of type [A-Z]+ (closed|opened)', r'\1 \2'),
        (r'\b(Tab [A-Z][a-z]*) (closed|opened) by [A-Za-z]+', r'\1 \2'),
        (r'\b(Tab [A-Z][a-z]*) (closed|opened)', r'\1 \2'),
        (r'Tab "(.*?)" of type BLOB', r'\1'),
    ]

    # Apply patterns to shorten the description
    for pattern, replacement in patterns:
        description = re.sub(pattern, replacement, description)

    return description

# Plots the most common activities in a project based on the filtered data
def common_activities_in_project(filters_dict, data):
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(data)

    # Filter the DataFrame based on the 'Project Name'
    df = df[df['Document'] == filters_dict['Project Name']]

    # Count occurrences of each activity description
    activities = df['Description'].value_counts()

    # Calculate average occurrence
    average = activities.mean()

    # Filter activities with counts >= 2 * average
    activities = activities[activities >= 2 * average]

    # Shorten activity descriptions
    activities.index = activities.index.map(shorten_description)

    # Wrap long labels
    wrapped_labels = [textwrap.fill(label, 10) for label in activities.index]

    # Prepare data for plotting
    activities = dict(zip(wrapped_labels, activities.values))

    # Generate bright colors for plotting
    colors = [(random.uniform(0.4, 1), random.uniform(0.4, 1), random.uniform(0.4, 1)) for _ in range(len(activities))]

    # Create plot using squarify
    fig, ax = plt.subplots()
    squarify.plot(sizes=activities.values(), label=activities.keys(), color=colors, ax=ax, alpha=0.7)

    # Adjust text labels size
    for label in ax.texts:
        label.set_fontsize(8)

    plt.title('Activities in Project "' + filters_dict['Project Name'] + '"')
    plt.axis('off')
    plt.tight_layout()

    return plt

In [32]:
# Creates a UI component that displays the chosen filters
def filters_data():
  filters_hboxes = [widgets.HBox([title := widgets.Label(value='Chosen Filters', layout=widgets.Layout(margin='0 auto', font_size='16px'))])]
  title.add_class('title')
  for key, value in filters_dict.items():
    if key is None or value is None:
      continue
    if key == 'Actions Type':
      value = 'Positive' if value else 'All'
    hbox = widgets.HBox([widgets.Label(value=key), widgets.Label(value=str(value), layout=widgets.Layout(margin="0 auto"))])
    filters_hboxes.append(hbox)

  filters = widgets.VBox(filters_hboxes, layout=widgets.Layout(margin="150px auto", height="75%"))
  filters.add_class('filters')
  filters.add_class('component')
  return filters

# Generates and displays a graph based on the selected filters
def create_graph(filters_dict, data):
  graph_output = widgets.Output()
  plt = None
  error = widgets.Label('Graph cannot be created')
  error.add_class('error')
  with graph_output:
    match filters_dict['Graph']:
      case 'AQ':
        plt = activities_quantity(filters_dict, data)
      case 'WH':
        plt = work_hours(filters_dict, data)
      case 'RW':
        plt = relative_work(filters_dict, data)
      case 'CA':
        plt = common_activities_in_project(filters_dict, data)
    if plt is not None:
      plt.show()
    else:
      display(error)

  graph = widgets.VBox([graph_output], layout=widgets.Layout(margin="90px auto", height="75%"))
  graph.add_class('component')
  return graph

# Creates the results screen based on the filtered parameters
def create_parameters_results_screen(filters_dict, data):
  screen_title = widgets.Label(value="Parameters Screen", layout=widgets.Layout(height="100%"))
  screen_title.add_class('screen-title')
  screen_hbox = widgets.HBox([filters_data(), create_graph(filters_dict, data)])
  screen = widgets.VBox([screen_title, screen_hbox])
  screen.add_class('par-screen')
  return screen

##Statistics Screen##

In [33]:
# Function to filter data based on user selections
def filter_data(df, start_date, end_date, user):
    if start_date:
        start_date = pd.to_datetime(start_date)
        df = df[df['Time'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date)
        df = df[df['Time'] <= end_date]
    if user != 'All':
        df = df[df['User'] == user]
    return df

# Function to add value labels to a bar graph
def add_value_labels(ax, spacing=5):
    for rect in ax.patches:
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2
        label = f"{y_value:.2f}"
        va = 'bottom' if y_value >= 0 else 'top'
        ax.annotate(label, (x_value, y_value), xytext=(0, spacing), textcoords="offset points", ha='center', va=va)

# Function to export data to Excel
def export_to_excel(data, filename):
    data.to_excel(filename, index=False)
    files.download(filename)

# Function to show negative actions
def show_negative_actions(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph displays the number of negative actions (deletions, closures, cancellations) performed by each user.")
        negative_actions = filtered_df[filtered_df['Description'].str.contains('Delete|Close|Cancel', case=False)]  # Filter for negative actions
        negative_count = negative_actions['User'].value_counts()  # Count the number of negative actions by user

        fig, ax = plt.subplots(figsize=(10, 6))
        negative_count.plot(kind='bar', ax=ax)
        add_value_labels(ax)

        plt.title('Number of Negative Actions by User')
        plt.xlabel('User')
        plt.ylabel('Number of Negative Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(negative_count.reset_index(name='Negative Actions'), '/content/negative_actions_data.xlsx'))

# Function to show project statistics
def show_project_stats(filtered_df, days=7):
    output.clear_output(wait=True)
    with output:
        print("This section provides an overview of the project statistics, including the duration, total actions, number of unique tabs, and number of unique users.")

        project_duration = filtered_df['Time'].max() - filtered_df['Time'].min()
        total_actions = len(filtered_df)
        unique_tabs = filtered_df['Tab'].nunique()  # Count the number of unique tabs
        unique_users = filtered_df['User'].nunique()  # Count the number of unique users

        # Average actions per day
        days_active = (filtered_df['Time'].max() - filtered_df['Time'].min()).days + 1
        avg_actions_per_day = total_actions / days_active

        # Percentage of negative actions
        negative_actions = filtered_df[filtered_df['Description'].str.contains('Delete|Close|Cancel', case=False)]
        percent_negative_actions = (len(negative_actions) / total_actions) * 100

        # Percentage of active users
        active_users = filtered_df['User'].nunique()
        total_users = df['User'].nunique()
        percent_active_users = (active_users / total_users) * 100

        # Most active day
        filtered_df.loc[:, 'Date'] = filtered_df['Time'].dt.date # Convert 'Time' column to 'Date' column
        actions_per_day = filtered_df.groupby('Date').size() # Count the number of actions per day
        most_active_day = actions_per_day.idxmax() # Get the date with the highest number of actions
        most_active_day_count = actions_per_day.max() # Get the number of actions on the most active day

        # Least active day
        least_active_day = actions_per_day.idxmin()
        least_active_day_count = actions_per_day.min()

        # Peak hour of activity
        filtered_df.loc[:, 'Hour'] = filtered_df['Time'].dt.hour
        actions_per_hour = filtered_df.groupby('Hour').size()
        peak_hour = actions_per_hour.idxmax()
        peak_hour_count = actions_per_hour.max()

        # Unique documents
        unique_documents = filtered_df['Document'].nunique()

        recent_date = filtered_df['Time'].max() - timedelta(days=days)  # Get the date 'days' days ago
        active_docs = filtered_df[filtered_df['Time'] > recent_date]['Document'].unique()  # Get the list of active documents
        inactive_docs = set(filtered_df['Document'].unique()) - set(active_docs)

        stats_html = f"""
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center; color: black;">
            <h2 style="color: #4A4A4A;">Project Statistics</h2>
            <p><strong>Project Duration:</strong> {project_duration}</p>
            <p><strong>Total Actions:</strong> {total_actions}</p>
            <p><strong>Number of Unique Tabs:</strong> {unique_tabs}</p>
            <p><strong>Number of Unique Users:</strong> {unique_users}</p>
            <p><strong>Average Actions Per Day:</strong> {avg_actions_per_day:.2f}</p>
            <p><strong>Percentage of Negative Actions:</strong> {percent_negative_actions:.2f}%</p>
            <p><strong>Percentage of Active Users:</strong> {percent_active_users:.2f}%</p>
            <p><strong>Most Active Day:</strong> {most_active_day} ({most_active_day_count} actions)</p>
            <p><strong>Least Active Day:</strong> {least_active_day} ({least_active_day_count} actions)</p>
            <p><strong>Peak Hour of Activity:</strong> {peak_hour}:00 ({peak_hour_count} actions)</p>
            <p><strong>Unique Documents:</strong> {unique_documents}</p>
            <p><strong>Active Documents (Last {days} Days):</strong> {', '.join(active_docs)}</p>
            <p><strong>Inactive Documents:</strong> {', '.join(inactive_docs)}</p>
        </div>
        """
        display(widgets.HTML(value=stats_html))

    project_stats_data = {
        'Metric': [
            'Project Duration',
            'Total Actions',
            'Number of Unique Tabs',
            'Number of Unique Users',
            'Average Actions Per Day',
            'Percentage of Negative Actions',
            'Percentage of Active Users',
            'Most Active Day',
            'Most Active Day Count',
            'Least Active Day',
            'Least Active Day Count',
            'Peak Hour of Activity',
            'Peak Hour Count',
            'Unique Documents',
            'Active Documents (Last 7 Days)',
            'Inactive Documents'
        ],
        'Value': [
            project_duration,
            total_actions,
            unique_tabs,
            unique_users,
            avg_actions_per_day,
            percent_negative_actions,
            percent_active_users,
            most_active_day,
            most_active_day_count,
            least_active_day,
            least_active_day_count,
            peak_hour,
            peak_hour_count,
            unique_documents,
            ', '.join(active_docs),
            ', '.join(inactive_docs)
        ]
    }
    export_button.on_click(lambda b: export_to_excel(pd.DataFrame(project_stats_data), '/content/project_stats_data.xlsx'))


# Function to show user progress
def show_user_progress(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph shows the cumulative number of actions performed by each user over time, indicating their activity trends.")
        filtered_df['Date'] = filtered_df['Time'].dt.date
        user_actions = filtered_df.groupby(['User', 'Date']).size().unstack(fill_value=0).cumsum(axis=1)  # Cumulative sum of actions
        plt.figure(figsize=(12, 6))
        for user in user_actions.index:
            plt.plot(user_actions.columns, user_actions.loc[user], label=user)
        plt.title('Cumulative Actions by User Over Time')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Actions')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(user_actions.reset_index(), '/content/user_progress_data.xlsx'))

# Function to show user effectiveness
def show_user_effectiveness(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph displays the average number of daily actions for each user, providing insights into their activity levels.")
        daily_actions = filtered_df.groupby(['User', filtered_df['Time'].dt.date]).size().reset_index(name='Actions') # Group by user and date
        avg_daily_actions = daily_actions.groupby('User')['Actions'].mean().sort_values(ascending=False) # Average daily actions

        fig, ax = plt.subplots(figsize=(10, 6))
        avg_daily_actions.plot(kind='bar', ax=ax)
        add_value_labels(ax)

        plt.title('Average Daily Actions by User')
        plt.xlabel('User')
        plt.ylabel('Average Daily Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(avg_daily_actions.reset_index(name='Average Daily Actions'), '/content/user_effectiveness_data.xlsx'))

# Function to show action types
def show_action_types(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph displays the most common types of actions performed in the project, highlighting the primary activities.")
        action_types = filtered_df['Description'].value_counts()
        plt.figure(figsize=(12, 6))
        action_types.head(10).plot(kind='bar')
        plt.title('Top 10 Most Common Action Types')
        plt.xlabel('Action Type')
        plt.ylabel('Count')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

        top_actions_html = f"""
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center;">
            <h2 style="color: #4A4A4A;">Top 10 Most Common Action Types</h2>
            <ul style="display: inline-block; text-align: left;">
                {''.join([f'<li>{action}</li>' for action in action_types.head(10).index])}
            </ul>
        </div>
        """
        display(widgets.HTML(value=top_actions_html))

    export_button.on_click(lambda b: export_to_excel(action_types.head(10).reset_index(name='Count'), '/content/action_types_data.xlsx'))

# Function to show activity by day of week
def activity_by_day_of_week(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This graph shows the distribution of activity across different days of the week, providing insights into weekly activity patterns.")
        filtered_df['Day'] = filtered_df['Time'].dt.day_name()
        day_counts = filtered_df['Day'].value_counts().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']) # Reorder days
        plt.figure(figsize=(10, 6))
        day_counts.plot(kind='bar')
        plt.title('Activity by Day of Week')
        plt.xlabel('Day of Week')
        plt.ylabel('Number of Actions')
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(day_counts.reset_index(name='Count'), '/content/activity_by_day_of_week_data.xlsx'))

# Function to show action type ratio
def action_type_ratio(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This pie chart shows the ratio of different types of actions (Create, Edit, Delete, Add) in the project.")
        action_types = filtered_df['Description'].apply(lambda x: 'Create' if 'Create' in x else ('Edit' if 'Edit' in x else ('Delete' if 'Delete' in x else 'Add')))
        type_counts = action_types.value_counts()
        type_counts = type_counts[type_counts.index.isin(['Create', 'Edit', 'Delete', 'Add', 'Other'])] # Reorder types
        plt.figure(figsize=(8, 8))
        plt.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
        plt.title('Ratio of Action Types')
        plt.axis('equal')
        plt.show()

    export_button.on_click(lambda b: export_to_excel(type_counts.reset_index(name='Count'), '/content/action_type_ratio_data.xlsx'))

# Function to show user collaboration
def user_collaboration(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This network graph shows how users are connected through their work on shared documents, indicating collaboration levels.")
        user_doc = filtered_df.groupby('Document')['User'].unique().apply(list) # Group by document and convert to list of users
        G = nx.Graph()
        for users in user_doc:
            for u1 in users:
                for u2 in users:
                    if u1 != u2:
                        if G.has_edge(u1, u2): # If edge already exists, increment weight by 1
                            G[u1][u2]['weight'] += 1
                        else:
                            G.add_edge(u1, u2, weight=1) # If edge doesn't exist, add it with weight 1
        pos = nx.spring_layout(G)
        plt.figure(figsize=(12, 8))
        nx.draw(G, pos, with_labels=True, node_color='lightblue',
                node_size=500, font_size=8, font_weight='bold')
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        plt.title('User Collaboration Network')
        plt.axis('off')
        plt.show()

    export_button.on_click(lambda b: export_to_excel(filtered_df[['User', 'Document']].drop_duplicates(), '/content/user_collaboration_data.xlsx'))

# Function to calculate effective work time
def calculate_effective_work_time(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis calculates the effective work time for each user, considering breaks longer than 30 minutes as non-work time.")
        effective_times = {}
        for user in filtered_df['User'].unique():
            user_actions = filtered_df[filtered_df['User'] == user].sort_values('Time')
            total_time = 0
            for i in range(1, len(user_actions)):
                time_diff = (user_actions.iloc[i]['Time'] - user_actions.iloc[i-1]['Time']).total_seconds() / 60
                if time_diff <= 30:  # if time difference is less than 30 minutes, consider it work time
                    total_time += time_diff
            effective_times[user] = total_time / 60  # convert to hours

        plt.figure(figsize=(10, 6))
        plt.bar(effective_times.keys(), effective_times.values())
        plt.title('Effective Work Time per User')
        plt.xlabel('User')
        plt.ylabel('Effective Work Time (Hours)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(pd.DataFrame(list(effective_times.items()), columns=['User', 'Effective Work Time']), '/content/effective_work_time_data.xlsx'))

# Function to analyze trends over time
def analyze_trends_over_time(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis shows how the number of actions changes over the course of the project, indicating activity trends.")
        filtered_df['Date'] = filtered_df['Time'].dt.date
        daily_actions = filtered_df.groupby('Date').size().reset_index(name='Actions') # Group by date and count actions

        plt.figure(figsize=(12, 6))
        plt.plot(daily_actions['Date'], daily_actions['Actions'])
        plt.title('Number of Actions Over Time')
        plt.xlabel('Date')
        plt.ylabel('Number of Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    export_button.on_click(lambda b: export_to_excel(daily_actions, '/content/trends_over_time_data.xlsx'))

# Function to analyze work styles
def analyze_work_styles(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis categorizes users based on their work patterns, providing insights into their work styles. 'Late Night' represents actions performed between 22:00 and 05:00, 'Short Bursts' represents actions performed within 5 minutes of the previous action, 'Weekend' represents actions performed on weekends, and 'Regular' represents actions not falling into the above categories.")
        user_patterns = {}
        for user in filtered_df['User'].unique():
            user_actions = filtered_df[filtered_df['User'] == user]
            total_actions = user_actions.shape[0]

            # Late night work (22:00-05:00)
            late_night = user_actions[user_actions['Time'].dt.hour.isin([22, 23, 0, 1, 2, 3, 4])].shape[0]

            # Short work bursts (less than 5 minutes between actions)
            short_bursts = sum(user_actions['Time'].diff().dt.total_seconds() < 300)

            # Weekend work
            weekend_work = user_actions[user_actions['Time'].dt.dayofweek.isin([5, 6])].shape[0]

            user_patterns[user] = {
                'Late Night': late_night / total_actions,
                'Short Bursts': short_bursts / total_actions,
                'Weekend': weekend_work / total_actions,
                'Regular': max(0, 1 - (late_night + short_bursts + weekend_work) / total_actions)  # remaining time
            }

        patterns_df = pd.DataFrame(user_patterns).T

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

        # Bar graph
        patterns_df.plot(kind='bar', stacked=True, ax=ax1)
        ax1.set_title('Work Styles of Users (Proportions)')
        ax1.set_xlabel('User')
        ax1.set_ylabel('Proportion of Actions')
        ax1.legend(title='Work Style', bbox_to_anchor=(1.05, 1), loc='upper left')

        # Table with exact data
        cell_text = []
        for user, data in patterns_df.iterrows():
            cell_text.append([f"{value:.2%}" for value in data])

        ax2.axis('tight')
        ax2.axis('off')
        table = ax2.table(cellText=cell_text, rowLabels=patterns_df.index, colLabels=patterns_df.columns,
                          cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.5)
        ax2.set_title('Work Styles of Users (Exact Percentages)')

        plt.tight_layout()
        plt.show()

        explanation_html = """
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center; color: black;">
            <h2 style="color: #4A4A4A;">Explanation of Work Styles</h2>
            <p><strong>Late Night:</strong> Proportion of actions performed between 22:00 and 05:00</p>
            <p><strong>Short Bursts:</strong> Proportion of actions performed within 5 minutes of the previous action</p>
            <p><strong>Weekend:</strong> Proportion of actions performed on weekends</p>
            <p><strong>Regular:</strong> Proportion of actions not falling into the above categories</p>
        </div>
        """
        display(widgets.HTML(value=explanation_html))

    export_button.on_click(lambda b: export_to_excel(patterns_df.reset_index(), '/content/work_styles_data.xlsx'))

# Function to analyze user contributions
def analyze_user_contributions(filtered_df):
    output.clear_output(wait=True)
    with output:
        print("This analysis shows the contribution of each user to the progress of the task.")
        # Explanation of the graph and the calculation
        explanation_html = """
        <div style="font-family: Arial, sans-serif; padding: 20px; text-align: center;">
            <h2 style="color: #4A4A4A;">User Contributions to Task Progress</h2>
            <p>This graph shows the contribution of each user to the progress of the task. The contribution is measured by the number of positive actions performed by each user.</p>
            <p><strong>Calculation Method:</strong></p>
            <ul style="text-align: left; display: inline-block;">
                <li>We count the number of positive actions each user performed within the selected time range.</li>
                <li>Positive actions include creating, editing, and adding significant content.</li>
                <li>The more positive actions a user performs, the higher their contribution to the task progress is considered.</li>
            </ul>
        </div>
        """
        display(widgets.HTML(value=explanation_html))

        # Define positive contribution actions
        positive_actions = ['Create', 'Edit', 'Add']

        # Filter actions that are considered positive contributions
        contribution_df = filtered_df[filtered_df['Description'].str.contains('|'.join(positive_actions), case=False)]

        # Calculate the number of positive contribution actions each user performed
        user_contributions = contribution_df.groupby('User')['Description'].count().sort_values(ascending=False)

        # Plot the contributions
        fig, ax = plt.subplots(figsize=(12, 6))
        user_contributions.plot(kind='bar', ax=ax)
        add_value_labels(ax)

        plt.title('User Contributions to Task Progress')
        plt.xlabel('User')
        plt.ylabel('Number of Positive Contribution Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


    # Enable export to Excel
    export_button.on_click(lambda b: export_to_excel(user_contributions.reset_index(name='Positive Contributions'), '/content/user_contributions_data.xlsx'))

# Function to update statistics based on selected filters
def update_statistics(statistic, start_date, end_date, user):
    filtered_df = filter_data(df, start_date, end_date, user)
    if statistic == 'Negative Actions':
        show_negative_actions(filtered_df)
    elif statistic == 'Project Stats':
        show_project_stats(filtered_df)
    elif statistic == 'User Progress':
        show_user_progress(filtered_df)
    elif statistic == 'User Effectiveness':
        show_user_effectiveness(filtered_df)
    elif statistic == 'Action Types':
        show_action_types(filtered_df)
    elif statistic == 'Activity by Day of Week':
        activity_by_day_of_week(filtered_df)
    elif statistic == 'Action Type Ratio':
        action_type_ratio(filtered_df)
    elif statistic == 'User Collaboration':
        user_collaboration(filtered_df)
    elif statistic == 'Effective Work Time':
        calculate_effective_work_time(filtered_df)
    elif statistic == 'Trends Over Time':
        analyze_trends_over_time(filtered_df)
    elif statistic == 'Work Styles':
        analyze_work_styles(filtered_df)
    elif statistic == 'User Contributions':
        analyze_user_contributions(filtered_df)

# Function to handle button click
def on_button_click(b):
    update_statistics(statistic_widget.value, start_date_widget.value, end_date_widget.value, user_widget.value)


# Function to create the screen
def create_statistics_screen():
  global df, min_date, max_date, statistic_widget, start_date_widget, end_date_widget, user_widget, export_button, output
  # Load the data
  df = pd.DataFrame(openJsonFromDB())

  # Convert 'Time' to datetime and sort by time
  df['Time'] = pd.to_datetime(df['Time'])
  df = df.sort_values('Time')

  # Get the minimum and maximum dates from the data
  min_date = df['Time'].min().date()
  max_date = df['Time'].max().date()
  # UI Design
  title_html = """
  <div style="background-color: #99b3ff; color: white; padding: 20px; text-align: center;">
      <h1 style="font-family: Arial, sans-serif;">Data Analysis System</h1>
  </div>
  """

  # Widget for selecting statistics
  statistic_widget = widgets.Dropdown(
      options=['Negative Actions', 'Project Stats',
             'User Progress', 'User Effectiveness', 'Action Types',
             'Activity by Day of Week', 'Action Type Ratio',
             'User Collaboration',
             'Effective Work Time', 'Trends Over Time', 'Work Styles','User Contributions'],
      value='Negative Actions',
      description='Select Statistic:',
      style={'description_width': 'initial'},
      layout=widgets.Layout(width='20%', margin='0 10px 0 0')
  )

  # Widget for selecting start date
  start_date_widget = widgets.DatePicker(
      description='Start Date:',
      disabled=False,
      style={'description_width': 'initial'},
      layout=widgets.Layout(width='20%', margin='0 10px 0 0'),
      value=min_date
  )

  # Widget for selecting end date
  end_date_widget = widgets.DatePicker(
      description='End Date:',
      disabled=False,
      style={'description_width': 'initial'},
      layout=widgets.Layout(width='20%', margin='0 10px 0 0'),
      value=max_date
  )

  # Widget for selecting user
  user_widget = widgets.Dropdown(
      options=['All'] + list(df['User'].unique()),
      value='All',
      description='Select User:',
      style={'description_width': 'initial'},
      layout=widgets.Layout(width='20%', margin='0 10px 0 0')
  )

  # Button for loading the selected filters
  load_button = widgets.Button(
      description='Load Selection',
      button_style='success',
      layout=widgets.Layout(width='20%', margin='10px 0 0 0')
  )

  # Button for exporting data
  export_button = widgets.Button(
      description='Export Data',
      button_style='info',
      layout=widgets.Layout(width='20%', margin='10px 0 0 0')
  )

  # Output widget to display graphs
  output = widgets.Output()
  output.add_class('statistics-output')


  load_button.on_click(on_button_click)

  # Styling the widgets
  statistic_widget.style.description_width = 'initial'
  start_date_widget.style.description_width = 'initial'
  end_date_widget.style.description_width = 'initial'
  user_widget.style.description_width = 'initial'
  load_button.style.button_color = '#4CAF50'
  export_button.style.button_color = '#2196F3'

  statistics_screen = widgets.VBox([widgets.HTML(value=title_html), HBox([statistic_widget, start_date_widget, end_date_widget, user_widget]), HBox([load_button, export_button]), output], layout=widgets.Layout(height="850px"))
  statistics_screen.add_class('statistics-screen')
  return statistics_screen

##Index Search Screen##

In [34]:
# Creates the index search screen UI for the application
def create_index_search_screen():
    # Create the headline
    screen_title = widgets.Label(
        value="Search by Index",
        layout=widgets.Layout(height="auto", width="100%")
    )
    screen_title.add_class('screen-title')

    # Create search input
    search_input = widgets.Text(
        value='',
        placeholder='Enter search term',
        disabled=False
    )
    search_input.add_class('centered-widget')

    # Create search button
    search_button = widgets.Button(
        description='Search',
        disabled=False,
        button_style='',
        tooltip='Click to search',
        icon='search'
    )

    # Create output area for search results
    search_output = widgets.HTML(
        value='<p style="color:black;">Search results will appear here</p>',
        layout=widgets.Layout(
            border='1px solid black',
            padding='10px',
            margin='10px 0px',
            max_height='300px',
            overflow_y='auto',
            width='80%'
        )
    )
    search_output.add_class('output')

    # Sample data for demonstration
    sample_data = get_index()

    def on_search_clicked(b):
        search_term = search_input.value.lower()
        results = []
        for index, count in sample_data.items():
            if search_term in index:
                results.append((index, count))

        if results:
            output = "<h3 style=\"color:black;\">Search Results:</h3>"
            for index, count in results:
                output += f"""
                <div style="margin-bottom: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; color: black;">
                    <h4 style="color: black;">{index.capitalize()}</h4>
                    <p style="color:black;"><strong>Count:</strong> {count}</p>
                </div>
                """
        else:
            output = f"<p style=\"color:black;\">No results found for '{search_term}'.</p>"

        search_output.value = output

    search_button.on_click(on_search_clicked)

    # Create a horizontal box layout for the search input and button
    search_hbox = widgets.HBox(
        [search_input, search_button],
        layout=widgets.Layout(justify_content='center', align_items='center', margin='0 auto')
    )

    # Create a vertical box layout for the search input/button and output
    content_vbox = widgets.VBox(
        [search_hbox, search_output],
        layout=widgets.Layout(align_items='center', justify_content='center', width='80%')
    )

    # Create the main screen layout with a top-aligned headline
    screen = widgets.VBox(
        [screen_title, content_vbox],
        layout=widgets.Layout(
            align_items='center',
            justify_content='flex-start',
            width='100%',
            height='100vh',
            padding='10px',
            margin='0 auto'
        )
    )
    screen.add_class('index-screen')
    return screen

##JSON Upload Screen##

In [35]:
# Creates the JSON upload screen UI for the application
def create_json_upload_screen():
    screen_title = widgets.Label(value="Manager Screen", layout=widgets.Layout(height="auto", width="100%"))
    screen_title.add_class('screen-title')

    # Create a file upload widget
    file_upload = widgets.FileUpload(
        accept='.json',
        multiple=False,
        description='Upload JSON',
        button_style='primary'
    )
    file_upload.add_class('centered-widget')

    upload_output = widgets.Output()

    header_area = widgets.Textarea(
        value='',
        description='Header:',
        disabled=False,
        layout=widgets.Layout(width='120%', height='60px', font_size='30px')
    )
    header_area.add_class('custom-header-textarea')

    paragraph_area = widgets.Textarea(
        value='',
        description='Text:',
        disabled=False,
        layout=widgets.Layout(width='120%', height='60px', font_size='30px')
    )
    paragraph_area.add_class('custom-paragraph-textarea')

    json_display_area = widgets.Textarea(
        value='',
        description='JSON Content:',
        disabled=True,
        layout=widgets.Layout(width='50%', height='300px', font_size='12px'),
        style={'font_family': 'monospace'}
    )
    json_display_area.add_class('component')

    next_button = widgets.Button(
        description='Next',
        button_style='',
        layout=widgets.Layout(width='50px', height='50px', font_size='20px', margin='10px')
    )

    back_button = widgets.Button(
        description='Back',
        button_style='',
        layout=widgets.Layout(width='50px', height='50px', font_size='20px', margin='10px')
    )

    button_box = widgets.HBox([back_button, next_button], layout=widgets.Layout(justify_content='center', width='70%'))

    content_display = widgets.VBox([header_area, paragraph_area, button_box], layout=widgets.Layout(align_items='center'))

    current_index = [0]  # Using a list to store mutable state
    headers_and_paragraphs = []

    def on_next_click(b):
        if current_index[0] < len(headers_and_paragraphs) - 1:
            current_index[0] += 1
            header, paragraph = headers_and_paragraphs[current_index[0]]
            header_area.value = header
            paragraph_area.value = paragraph
        else:
            header_area.value = "End of headers"
            paragraph_area.value = "End of paragraphs"

    def on_back_click(b):
        if current_index[0] > 0:
            current_index[0] -= 1
            header, paragraph = headers_and_paragraphs[current_index[0]]
            header_area.value = header
            paragraph_area.value = paragraph
        else:
            header_area.value = "Start of headers"
            paragraph_area.value = "Start of paragraphs"

    next_button.on_click(on_next_click)
    back_button.on_click(on_back_click)

    def on_file_upload(change):
        nonlocal headers_and_paragraphs
        with upload_output:
            upload_output.clear_output()
            for fname, file_info in file_upload.value.items():
                content = file_info['content']
                print(f'Uploaded file name: {fname}')

                try:
                    json_data = json.loads(content.decode("utf-8")) # Load JSON data

                    if not isinstance(json_data, list):
                        json_data = [json_data]  # Convert to list if it's not already

                    saveJsonToDB(json_data) # Save JSON data to database

                    headers_and_paragraphs = []
                    for item in json_data:
                        if isinstance(item, dict):
                            for key, value in item.items():
                                headers_and_paragraphs.append((key, str(value)))
                        elif isinstance(item, str):
                            # Assume the entire string is a paragraph without a header
                            headers_and_paragraphs.append(("Paragraph", item))
                        else:
                            headers_and_paragraphs.append(("Unknown", str(item)))

                    current_index[0] = 0  # Reset to first header and paragraph
                    if headers_and_paragraphs:
                        header, paragraph = headers_and_paragraphs[0]
                        header_area.value = header
                        paragraph_area.value = paragraph
                    else:
                        header_area.value = "No headers found"
                        paragraph_area.value = "No paragraphs found in the JSON file"

                    # Update JSON display area
                    json_display_area.value = json.dumps(json_data, indent=4, ensure_ascii=False)

                except json.JSONDecodeError:
                    header_area.value = 'Error: Invalid JSON file'
                    paragraph_area.value = ''
                    json_display_area.value = ''

    file_upload.observe(on_file_upload, names='value')

    screen_hbox = widgets.HBox(
        [file_upload, upload_output],
        layout=widgets.Layout(justify_content='center', align_items='center')
    )
    screen_hbox.add_class('json-screen')

    main_hbox = widgets.HBox(
        [content_display, json_display_area],
        layout=widgets.Layout(justify_content='center', align_items='flex-start', width='100%')
    )
    main_hbox.add_class('json-screen')

    manager_screen = widgets.VBox(
        [screen_title, screen_hbox, main_hbox],
        layout=widgets.Layout(align_items='center', justify_content='center', width='100%')
    )
    manager_screen.add_class('json-screen')
    return manager_screen

##ChatBot Screen##

In [36]:
# Retrieves and lists all users from the chatbot data
def getUsers():
  users_list = dataChatBot['User'].unique().tolist()
  response = "\n"
  for user in users_list:
    response += f"{user}\n"
  return response

In [37]:
# Returns the counts of the most frequent actions based on user input
def getCountsOfTopActions(user_input):
  split_user_input = user_input.split(' ')
  wanted_actions_number = 0
  for res in split_user_input:
    if res.isnumeric():
      wanted_actions_number = res
      break
  actions_dict = dataChatBot['Description'].value_counts().to_dict()
  sorted_actions = sorted(actions_dict.items(), key=lambda x: x[1], reverse=True)
  wanted_actions = sorted_actions[:int(wanted_actions_number)]
  response = "\n"
  for action, count in wanted_actions:
    response += f"{action}: {count}\n"
  return response

In [38]:
# Returns the latest date that appears in the data
def getLastDateOfProject():
  return str(dataChatBot['Time'].max())

In [39]:
# Finds the common actions that a user has made
def getActionsOfAUser(user_input):
  user_input = user_input.split(' ')
  users = dataChatBot['User'].unique().tolist()
  for res in user_input:
    res = res[:-1]
    if res in users:
      user_actions = dataChatBot[dataChatBot['User'] == res]['Description'].value_counts().to_dict()
      user_actions_mean = np.mean(list(user_actions.values()))
      response = "\n"
      for action, count in user_actions.items():
        if count > user_actions_mean:
          response += f"{action}\n"
      return response
  return "No user found"

In [40]:
# Returns the time difference between the start and the end of the project
def getDurationOfProject():
  return str(pd.to_datetime(dataChatBot['Time'].max()) - pd.to_datetime(dataChatBot['Time'].min()))

In [41]:
# Retrieves the amount of different documents that appear in the data
def getNumberOfDocuments():
  return dataChatBot['Document'].unique().size

In [42]:
# Returns the counters of each tab that appears in the data
def getTabsCountInProject(user_input):
  project_name = user_input[:-1]
  return len(dataChatBot[dataChatBot['Document'] == project_name]['Tab'].unique())

In [43]:
# Retrieves all users and identifies the most popular action each user has performed
# It returns a string listing each user with their most frequent action
def getUsersWithTheirMostPopularAction():
  users = dataChatBot['User'].unique().tolist()
  response = "\n"
  for user in users:
    user_actions = dataChatBot[dataChatBot['User'] == user]['Description'].value_counts().to_dict()
    most_popular_action = max(user_actions, key=user_actions.get)
    response += f"{user}: {most_popular_action}\n"
  return response

In [44]:
# Takes a list of patterns (questions) and returns a formatted string listing
# all available questions that the chatbot can answer
def getAllAvailableQuestions(patterns):
  response = "\n"
  for index, pattern in enumerate(patterns):
    response += f"{index + 1}. {pattern[0]}\n"
  return response

In [45]:
# Overrides the `respond` method of the `Chat` class to provide custom responses based on user input
# It matches the input against predefined patterns and returns an appropriate response
# If no match is found, it returns a default response
class CustomChat(Chat):
  def respond(self, user_input):
    for pattern, responses in self._pairs:
      match = re.match(pattern, user_input)
      if match:
        response = responses[0]
        if callable(response):
          return response(match)
        return response
    if user_input.lower() == 'exit' or user_input.lower() == 'bye':
      return "Bye!"
    return "I'm sorry, I didn't understand that."

In [46]:
# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatBot.', 'I go by the name ChatBot.']),
    (r'my name is (.*)', ['Glad to meet you.', 'Nice to meet you.']),
    (r'thank you (.*)', ['You\'re welcome!', 'Happy to help!']),
    (r'What is the purpose of the system?', ['The purpose of the system is to help software managers improve the teams performance by analyzing their work']),
    (r'whats included in the software', ['The software includes four screens to use: Parameters, Statistics, Index Searching and JSON Uploading']),
    (r'can you explain about the parameters screen?', ['The parameters screen allows you to choose which graph you want to see.']),
    (r'can you explain about the statistics screen?', ['The statistics screen allows you to see some interesting statistics.']),
    (r'can you explain about the index searching screen?', ['The index searching screen allows you to search for specific data from the OnShape index.']),
    (r'can you explain about the json uploading screen?', ['The json uploading screen allows you to upload a json file as the source for the graphs.']),
    (r'what is onshape|onshape|on shape', ['Onshape is a cloud-based CAD platform for product design and development. Learn more here: https://www.onshape.com/']),
    (r'how to add a new team member', ['At this moment of development we don\'t have this feature yet.']),
    (r'can you list all users?', [lambda match: getUsers()]),
    (r'can you list top (\d+) common actions?', [lambda match: getCountsOfTopActions(match.group(1))]),
    (r'what is the last date of a project?', [lambda match: getLastDateOfProject()]),
    (r'can you list common actions of (.*)?', [lambda match: getActionsOfAUser(match.group(1))]),
    (r'what is the duration of the project?', [lambda match: getDurationOfProject()]),
    (r'how many projects are there?', [lambda match: getNumberOfDocuments()]),
    (r'how many tabs are in the project (.+)?', [lambda match: getTabsCountInProject(match.group(1))]),
    (r'what are the most popular actions of each user?', [lambda match: getUsersWithTheirMostPopularAction()]),
]

# Create a chatbot
chatbot = CustomChat(patterns, reflections)

In [47]:
# Creates the chatbot UI screen
# It includes an input field for user messages, a send button, and a scrollable chat history display
# The chatbot interacts with the user based on predefined patterns and responses
def create_chatbot_screen():
  global dataChatBot
  dataChatBot = pd.DataFrame(openJsonFromDB())
  # Create widgets
  input_text = widgets.Text(
      value='',
      placeholder='Type your message here...',
      description='You:',
      disabled=False,
      layout=widgets.Layout(width='80%', margin="4px 0")
  )

  send_button = widgets.Button(
      description='Send',
      disabled=False,
      button_style='success',
      tooltip='Send message',
      icon='check',
      layout=widgets.Layout(width='10%', margin="4px 4px")
  )

  conversation_history = []

  initial_entry = widgets.HTML(f'<b style="color: black">Chatbot:</b> <span style="color: black">Here are all questions you can ask: </span><span style="color: black">{getAllAvailableQuestions(patterns)}</span>')
  chat_vbox = widgets.VBox([initial_entry])

  scrollable_chat = widgets.Box(children=[chat_vbox], layout=widgets.Layout(
      overflow_y='scroll',
      border='1px solid black',
      height='300px',
      width='95%',
      margin='4px auto'
  ))


  # Define the button click event handler
  def on_send_button_clicked(b):
      user_input = input_text.value
      if user_input.strip() != "":
          response = chatbot.respond(user_input)
          conversation_history.append((user_input, response))

          # Update the VBox with new chat entries
          chat_entries = [initial_entry]
          for user_msg, bot_msg in conversation_history:
              chat_entries.append(widgets.HTML(f'<b style="color: black">You:</b> <span style="color: black">{user_msg}</span>'))
              chat_entries.append(widgets.HTML(f'<b style="color: black">Chatbot:</b> <span style="color: black">{bot_msg}</span>'))

          chat_vbox.children = chat_entries
          input_text.value = ''  # Clear input text

  # Link button click event to handler
  send_button.on_click(on_send_button_clicked)

  hbox = widgets.HBox([input_text, send_button])

  chatbot_vbox = widgets.VBox([scrollable_chat, hbox])
  chatbot_vbox.add_class("chatbot-container")

  return chatbot_vbox

#Main Page#

In [48]:
# Handles the click event for the "Home" button, transitioning the user back to the home screen
def home_screen_clicked(b):
  page.children = [create_home_screen()]

In [49]:
# Handles the click event for the "Disconnect" button, logging the user out and returning to the login screen
def disconnect_button_clicked(b):
  page.children = [create_login_screen()]
  login_dict.clear()

In [50]:
# Handles the click event for the "Chatbot" button, transitioning the user to the chatbot screen
def chatbot_clicked(b):
  page.children = [create_menu(), create_chatbot_screen()]

In [51]:
# Creates the navigation menu for the application, allowing users to navigate
# between different screens, including Home, Parameters, Statistics,
# JSON Upload, Index Search, and Chatbot
# It also includes a disconnect button
def create_menu():
  home_screen = widgets.Button(description="Home")
  home_screen.on_click(home_screen_clicked)

  parameters_screen = widgets.Button(description="Parameters", layout=widgets.Layout(disabled=True))
  parameters_screen.on_click(parameters_clicked)

  statistics_screen = widgets.Button(description="Statistics", layout=widgets.Layout(disabled=True))
  statistics_screen.on_click(statistics_clicked)

  json_upload_screen = widgets.Button(description="JSON Upload", layout=widgets.Layout(disabled=True))
  json_upload_screen.on_click(json_upload_clicked)

  index_search_screen = widgets.Button(description="Index Search", layout=widgets.Layout(disabled=True))
  index_search_screen.on_click(index_search_clicked)

  chatbot_button = widgets.Button(description="Chatbot", layout=widgets.Layout(disabled=True))
  chatbot_button.on_click(chatbot_clicked)

  disconnect_button = widgets.Button(description="Disconnect", button_style="danger")
  disconnect_button.on_click(disconnect_button_clicked)

  menu_buttons_list.extend([parameters_screen, statistics_screen, json_upload_screen, index_search_screen])

  menu = widgets.HBox([home_screen, parameters_screen, statistics_screen, json_upload_screen, index_search_screen, chatbot_button, disconnect_button])
  menu.add_class('menu')
  return menu

In [52]:
# page is the main container for the application and its children will change
# depending on which screen the user chooses through the menu or the home screen
page = widgets.VBox(layout=widgets.Layout(width="100%", height="100%"))
page.add_class("page")
page.children = [create_login_screen()]

#Main Program#

In [53]:
# Connect to the Firebase database
FBConn = connect_to_DB(DB_url)
# Check if an index is already stored in the database
# If not, create the index and store it in the database
if FBConn.get('/onshapeIndex/', None) == None:
  post_index_to_DB()
# Check if the JSON file is already stored in the database
# If not, download it and store it in the database
if FBConn.get('/onShapeJSON/', None) == None:
  file_id = '16RcQ6BRIeYpKzassT2X5ezfTSIU5OVoH'
  url = f'https://drive.google.com/uc?id={file_id}'
  output = 'downloaded_file.json'
  gdown.download(url, output, quiet=False)
  with open(output, 'r') as json_file:
    json_data = json.load(json_file)
  saveJsonToDB(json_data)

In [54]:
# Display the page (the application)
display(page)

# Display the style.css stylesheet for some screen-specific styling
display(HTML(index_style))
display(HTML(par_style))
display(HTML(page_style))
display(HTML(json_style))